In [1]:
# import packages
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import os
import scipy
from sklearn.impute import SimpleImputer

In [2]:
# change viewing options
pd.set_option('display.max_columns', None)  
pd.set_option('max_colwidth', -1)

### Step 1: Import data

In [3]:
# read in the data
train = pd.read_csv("training_set_features.csv")
train_labels = pd.read_csv("training_set_labels.csv")
test = pd.read_csv("test_set_features.csv")

### Step 2: Do data preprocessing, scaling, etc.

From the data exploration phase, we decided which columns were relevant to our analysis. 

So, we'll subset the training and test sets by these features

In [4]:
# get columns to subset
columns_subset = ['h1n1_concern', 'h1n1_knowledge', 'doctor_recc_h1n1', 'doctor_recc_seasonal', 
                 'chronic_med_condition', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 
                 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 
                 'opinion_seas_sick_from_vacc', 'age_group', 'education', 'income_poverty']

In [5]:
# subset datasets
train_subset = train.loc[:, columns_subset]
test_subset = test.loc[:, columns_subset]

Now, we have to do something about any missing data that we observe:

In [6]:
# loop through all columns, count NaNs
print(f"The number of NaNs, by column, in the training set, is: \n\n{train_subset.isna().sum()} \n\n =========")
print(f"The number of NaNs, by column, in the test set, is: \n\n{test_subset.isna().sum()} \n\n")

The number of NaNs, by column, in the training set, is: 

h1n1_concern                   92  
h1n1_knowledge                 116 
doctor_recc_h1n1               2160
doctor_recc_seasonal           2160
chronic_med_condition          971 
opinion_h1n1_vacc_effective    391 
opinion_h1n1_risk              388 
opinion_h1n1_sick_from_vacc    395 
opinion_seas_vacc_effective    462 
opinion_seas_risk              514 
opinion_seas_sick_from_vacc    537 
age_group                      0   
education                      1407
income_poverty                 4423
dtype: int64 

The number of NaNs, by column, in the test set, is: 

h1n1_concern                   85  
h1n1_knowledge                 122 
doctor_recc_h1n1               2160
doctor_recc_seasonal           2160
chronic_med_condition          932 
opinion_h1n1_vacc_effective    398 
opinion_h1n1_risk              380 
opinion_h1n1_sick_from_vacc    375 
opinion_seas_vacc_effective    452 
opinion_seas_risk              499 
opinion_s

For the survey data, we can likely impute the NaNs with either 0/unknown or the mean/median values. For the demographic values that are unknown (education, income_poverty), we can just impute them with 'unknown'. Let's check the dtypes of each column. If they're all categorical/object, we can just impute missing data with "unknown"

In [7]:
# for training columns (test columns will be the same)
for col in train_subset.columns:
    print(f"The dtype of the {col} column is: {type(train_subset[col][0])}\n=======")

The dtype of the h1n1_concern column is: <class 'numpy.float64'>
The dtype of the h1n1_knowledge column is: <class 'numpy.float64'>
The dtype of the doctor_recc_h1n1 column is: <class 'numpy.float64'>
The dtype of the doctor_recc_seasonal column is: <class 'numpy.float64'>
The dtype of the chronic_med_condition column is: <class 'numpy.float64'>
The dtype of the opinion_h1n1_vacc_effective column is: <class 'numpy.float64'>
The dtype of the opinion_h1n1_risk column is: <class 'numpy.float64'>
The dtype of the opinion_h1n1_sick_from_vacc column is: <class 'numpy.float64'>
The dtype of the opinion_seas_vacc_effective column is: <class 'numpy.float64'>
The dtype of the opinion_seas_risk column is: <class 'numpy.float64'>
The dtype of the opinion_seas_sick_from_vacc column is: <class 'numpy.float64'>
The dtype of the age_group column is: <class 'str'>
The dtype of the education column is: <class 'str'>
The dtype of the income_poverty column is: <class 'str'>


Because the columns are a mix of ints and strings, we can split the data, impute each separately, then re-append them to each other

In [8]:
# split values
train_subset_numeric = train_subset.loc[:, ['h1n1_concern', 'h1n1_knowledge', 'doctor_recc_h1n1', 'doctor_recc_seasonal', 
                 'chronic_med_condition', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 
                 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 
                 'opinion_seas_sick_from_vacc']]

train_subset_str = train_subset.loc[:, [ 'age_group', 'education', 'income_poverty']]

test_subset_numeric = test_subset.loc[:, ['h1n1_concern', 'h1n1_knowledge', 'doctor_recc_h1n1', 'doctor_recc_seasonal', 
                 'chronic_med_condition', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 
                 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 
                 'opinion_seas_sick_from_vacc']]

test_subset_str = test_subset.loc[:, [ 'age_group', 'education', 'income_poverty']]


For the numeric values, there are several strategies that we could use:

1. Impute with mean/median: This would be the simplest strategy, but might create class imbalances and skew the results by artificially inflating some counts
2. Impute with predicted value (e.g., fit a logistic regression, and use the prediction as the impute value): this would probably be the most robust method, since it uses existing data to make a prediction on what the true values should be. But, this method is imperfect because we also have other missing data in other categories as well (so, to fit a logistic regression, we'd have to impute these other missing values) and we also don't know how reliable our imputations would be. 
3. Impute randomly, such that at the end, the class proportions remain the same. This method would keep the balance in proportions consistent, but at the cost of introducing noise by randomly assigning class values to observations. 
4. Filter out missing data.

Personally, I lean towards using a logistic regression to predict what the missing values should be, since it seems to be the method that will best maintain existing class proportions while accounting for the other features of the data (so, two people who have the same values for every other column in the dataset should presumably have the same value for the column in question)

But, before doing so, it's worth checking to see how "similar" the missing people are to those who we have data for. If they're similar, it would make sense to use information from the existing data in order to impute the missing data. Otherwise, it might be advisable to adapt the imputation method or filter them out entirely. 


#### Checking the similarity of missing data and non-missing data

For our data, we can use a two-sample Kolmogorov-Smirnov test, which tests whether two samples come from the same distribution. The null hypothesis is that the two samples come from the same distribution. A significant result (p<0.05) would tell us that the two samples come from a different distribution.

So, what we can do is loop through all the numeric columns that have NaNs. For each column, we take out that column from the dataframe. Then, we take the remaining data, split it into two sets, the first being the rows that had values for the column that we took out, the second being the rows that had NaNs for the column that we took out. We can then use the Kolmogorov-Smirnov test. A non-significant result is a good thing for us, since it means that the people who had missing values for that colummn "look like" the ones who didn't have missing values - therefore, we can use information from people who didn't have missing values, and make our predictions.

We run into an issue, which is that the scipy implementation of the KS test is univariate, when ideally we would want a multivariate implementation. So, we can't do a multivariate KS test in Python unless we recreate the algorithm ourselves.

We can try doing a univariate KS test on each column and then see the # of times that we get a significant difference. This is an imperfect method and using a multivariate KS test would be superior, but given the fact that this functionality doesn't exist in Python, I'll use it. I'll set an arbitrary threshold, where if the rows without NaNs and the rows with NaNs significantly differ in more than half of their columns, then their distributions significantly differ. 

If I were to redo this part of the analysis, I would implement the algorithm in the following document (https://www.sciencedirect.com/science/article/pii/S0167715297000205) or consider using the multivariate Cramer's test in R. 

In [17]:
for col in train_subset_numeric.columns:
    
    # get indices without NaNs
    indices_complete = train_subset_numeric[col].notnull()
    
    # take out column
    train_dropped = train_subset_numeric.drop([col], axis = 1)
    
    # get two datasets, one with NaNs and one without NaNs
    train_complete = train_dropped[indices_complete]
    train_NaNs = train_dropped[~indices_complete]
    
    # p-values list
    p_values = []
    # run KS-test for each column
    print(f"Running KS test for the {col} column:\n")
    for col2 in train_complete.columns:
        ks, p = scipy.stats.ks_2samp(train_complete[col2], train_NaNs[col2])   
        p_values.append(p)
    
    # get # of p-values less than 0.05
    num_values_less = len([x for x in p_values if x < 0.05])
    prop_values_less = round(num_values_less / len(p_values), 4)
    
    # print
    print(f"The proportions of significant comparisons, using the KS test, for the {col} column is: {prop_values_less}")
    print("===============================================")

Running KS test for the h1n1_concern column:

The proportions of significant comparisons, using the KS test, for the h1n1_concern column is: 0.6
Running KS test for the h1n1_knowledge column:

The proportions of significant comparisons, using the KS test, for the h1n1_knowledge column is: 0.2
Running KS test for the doctor_recc_h1n1 column:

The proportions of significant comparisons, using the KS test, for the doctor_recc_h1n1 column is: 1.0
Running KS test for the doctor_recc_seasonal column:

The proportions of significant comparisons, using the KS test, for the doctor_recc_seasonal column is: 1.0
Running KS test for the chronic_med_condition column:

The proportions of significant comparisons, using the KS test, for the chronic_med_condition column is: 1.0
Running KS test for the opinion_h1n1_vacc_effective column:

The proportions of significant comparisons, using the KS test, for the opinion_h1n1_vacc_effective column is: 1.0
Running KS test for the opinion_h1n1_risk column:

The

Based off these results, it seems like the univariate distributions for each column are significantly different. It seems like it would be hard to justify using a logistic regression given that the composition of the data is different for people with NaNs as opposed to those without NaNs. 

If I see this is an area that I can further study in order to improve model performance, I will re-consider using logistic regression. However, due to the results of the tests above, convenience, and efficiency, I'll impute using a different method.

Since the observations are categorical (even though they're encoded as numeric), it makes more sense to use the median as opposed to the mean. 

##### Impute the numeric data

For the numeric data, I'll impute the NaNs with the median value of the column

In [22]:
# create imputer objects
imp_train = SimpleImputer(missing_values=np.nan, strategy='median')
imp_test = SimpleImputer(missing_values=np.nan, strategy='median')

# fit imputer objects
train_subset_numeric_imputed = pd.DataFrame(imp_train.fit_transform(train_subset_numeric),
                                            columns = train_subset_numeric.columns)
test_subset_numeric_imputed = pd.DataFrame(imp_test.fit_transform(test_subset_numeric), 
                                           columns = test_subset_numeric.columns)

##### Impute categorical data

For the categorical data, I'll impute with a constant value ('unknown')

In [37]:
# create imputer objects
imp_train = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 'unknown')
imp_test = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 'unknown')

# fit imputer objects
train_subset_str_imputed = pd.DataFrame(imp_train.fit_transform(train_subset_str),
                                            columns = train_subset_str.columns)
train_subset_str_imputed = pd.DataFrame(imp_test.fit_transform(train_subset_str), 
                                           columns = train_subset_str.columns)

#### Re-join imputed data

Now that the data has been imputed, I'll re-join them

In [44]:
imputed_train = pd.concat([train_subset_numeric_imputed, train_subset_str_imputed], axis = 1)
imputed_test = pd.concat([test_subset_numeric_imputed, train_subset_str_imputed], axis = 1)

Now, the missing data has been imputed. We can start fitting models to this data. 